In [1]:
from google.colab import drive

drive.mount('/content/drive')

%cd /content/drive/MyDrive/ift6289-project/
!ls

Mounted at /content/drive
/content/drive/MyDrive/ift6289-project
'Copy of ift6289-project-facebook mbart-large-50-many-to-many-mmt-BNtoEN.ipynb'
'Copy of ift6289-project-mt5-base-ENtoBN.ipynb'
 gemma_7b_finetune_template.ipynb
 hasan-etal-2020-low
 hasan-etal-2020-low.tar.bz2
'ift6289-project-facebook mbart-large-50-many-to-many-mmt-BNtoEN.ipynb'
'ift6289-project-facebook mbart-large-50-many-to-many-mmt-ENtoBN.ipynb'
'ift6289-project-facebook nllb-200-distilled-600M.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-bn-en.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-en-mul.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-mul-en.ipynb'
'ift6289-project-meta-llama Llama-2-7b-hf.ipynb'
 ift6289-project-mt5-base-BNtoEN.ipynb
 ift6289-project-mt5-base-ENtoBN.ipynb
 ift6289-project-umt5-base-BNtoEN.ipynb
 ift6289-project-umt5-base-ENtoBN.ipynb
 logs
 results


In [2]:
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install transformers
!pip install ml_things
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 9.4 MB/s eta 0:00:00
  Created wheel for ml_things: filename=ml_things-0.0.1-py3-none-any.whl size=24393 sha256=81a47e6ab62a701508581b

In [3]:
from transformers import set_seed, AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, pipeline
from datasets import Dataset, load_dataset, load_metric
import torch
import pandas as pd
from tqdm.auto import tqdm  # tqdm is great for progress bars
import sacrebleu
import numpy as np

In [22]:
set_seed(42)

dataset = load_dataset("csebuetnlp/BanglaNMT")
# print(dataset)
# print(dataset['train'][0])
# print(dataset['train'][1])
# print(dataset['train'][2])
# print(dataset['train'][0]['bn'])
# print(dataset['train'][0]['en'])

training_subset_size = 100000
dataset["train"] = dataset["train"].select(range(training_subset_size))

In [23]:
# Specify the model checkpoint
# model_checkpoint = "facebook/nllb-200-distilled-600M"
model_checkpoint = "Helsinki-NLP/opus-mt-mul-en"
# model_checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [24]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples['bn'], max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    # Prepare the labels (Bengali translations)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['en'], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)
print(tokenized_datasets)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 597
    })
})


In [25]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [26]:
# Load both BLEU and SacreBLEU metrics
bleu_metric = load_metric('bleu')
sacrebleu_metric = load_metric('sacrebleu')

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    # # Compute BLEU score
    # bleu_result = bleu_metric.compute(predictions=decoded_preds, references=[decoded_labels])
    # Compute SacreBLEU score
    sacrebleu_result = sacrebleu_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Prepare result dictionary
    results = {
        # "bleu": bleu_result['score'],
        "sacrebleu": sacrebleu_result['score']
    }

    return results

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the 

In [27]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="sacrebleu",
    # report_to="none",
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# # Pre-training test results
baseline_test_results = trainer.predict(tokenized_datasets["test"], metric_key_prefix="test")
print("Baseline Test Metrics:", baseline_test_results.metrics)
baseline_validation_results = trainer.evaluate()
print("Baseline Validation Metrics:", baseline_validation_results)

# Fine-tune the model
print("\nFine-tuning the model...")
trainer.train()
print("Model fine-tuning complete.")

Baseline Test Metrics: {'test_loss': 5.578831672668457, 'test_sacrebleu': 14.397189477569036, 'test_runtime': 59.88, 'test_samples_per_second': 16.7, 'test_steps_per_second': 1.052}


Baseline Validation Metrics: {'eval_loss': 5.701390266418457, 'eval_sacrebleu': 13.822322698963632, 'eval_runtime': 32.593, 'eval_samples_per_second': 18.317, 'eval_steps_per_second': 1.166}

Fine-tuning the model...


Epoch,Training Loss,Validation Loss,Sacrebleu
1,0.377400,0.393021,19.304227
2,0.318100,0.367345,21.111832
3,0.294100,0.361596,21.634978


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[64171]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[64171]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

Model fine-tuning complete.


In [28]:
# Evaluate the model
validation_results = trainer.evaluate()
print("Final validation Metrics", validation_results)

Final validation Metrics {'eval_loss': 0.3615964651107788, 'eval_sacrebleu': 21.63497809691181, 'eval_runtime': 32.7573, 'eval_samples_per_second': 18.225, 'eval_steps_per_second': 1.16, 'epoch': 3.0}


# Test on RisingNews

In [29]:
# Calculate scores on the test set
final_test_results = trainer.predict(tokenized_datasets["test"], metric_key_prefix="test")
print("Final Test Metrics:", final_test_results.metrics)

Final Test Metrics: {'test_loss': 0.35946884751319885, 'test_sacrebleu': 22.111349670939457, 'test_runtime': 55.3412, 'test_samples_per_second': 18.07, 'test_steps_per_second': 1.138}


In [30]:
# Decode the predictions and labels
decoded_predictions = tokenizer.batch_decode(final_test_results.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(final_test_results.label_ids, skip_special_tokens=True)

# Print a sample of the predictions and labels
print("Sample Predictions and Labels:")
for i in range(10):  # Adjust the range to print more or fewer examples
    print(f"\nPrediction {i+1}: {decoded_predictions[i]}")
    print(f"Label {i+1}: {decoded_labels[i]}")

Sample Predictions and Labels:

Prediction 1: The senior officials of the Bangladesh High Commissioner at the airport, Sayida Muna Tasnim, and the Bangladesh Mission in London, announced his resignation.
Label 1: Bangladesh High Commissioner to the United Kingdom Saida Muna Tasneen and senior officials of Bangladesh Mission in London saw him off at the airport.

Prediction 2: Terrorism has no character, religion, and country.
Label 2: The terrorists have no race, religion and countries.

Prediction 3: Drake Cabera, professor of the University of Cornelius, wrote an article called "The Dark Side of Classed Thought."
Label 3: Derek Cabera, professor at Cornell University, wrote an essay called, "The dark sides of categorical thinking."

Prediction 4: He told the public that he would be able to buy rice oil at 45 crores per kg, rice at 50 crores, rice at 50 crores and saibin oil at 85 litres per litre.
Label 4: He said that people can buy onion at Tk 45 per kg, sugar at Tk 50 kg, lentil a

In [31]:
# Save predictions to a CSV file without column names
predictions_df = pd.DataFrame({'Prediction': decoded_predictions})
predictions_df.to_csv('results/helsinki_risingnews_predictions_mul_en.csv', index=False, header=False)

# Save labels to a separate CSV file without column names
labels_df = pd.DataFrame({'Label': decoded_labels})
labels_df.to_csv('results/helsinki_risingnews_labels_mul_en.csv', index=False, header=False)

# Test on SUPara-benchmark

In [32]:
supara_source_test_path = f"hasan-etal-2020-low/data/SUPara-benchmark/suparatest2018/suparatest2018_bn.txt"
supara_target_test_path = f"hasan-etal-2020-low/data/SUPara-benchmark/suparatest2018/suparatest2018_en.txt"

with open(supara_source_test_path, "r", encoding="utf-8") as f:
    supara_source_data = f.readlines()

with open(supara_target_test_path, "r", encoding="utf-8") as f:
    supara_target_data = f.readlines()

print(supara_source_data[0])
print(supara_target_data[0])

সে লাল গোলাপ পছন্দ করে।

She likes red roses.



In [33]:
supara_dataset = Dataset.from_dict({"bn": supara_source_data, "en": supara_target_data})
tokenized_supara_dataset = supara_dataset.map(preprocess_function, batched=True)
print(tokenized_supara_dataset)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [34]:
final_test_results_supara = trainer.predict(tokenized_supara_dataset, metric_key_prefix="test")
print("Final test metrics SUPara:", final_test_results_supara.metrics)
print("Final test results SUPara:", final_test_results_supara)

Final test metrics SUPara: {'test_loss': 0.42624175548553467, 'test_sacrebleu': 18.777902899595272, 'test_runtime': 23.7257, 'test_samples_per_second': 21.074, 'test_steps_per_second': 1.349}
Final test results SUPara: PredictionOutput(predictions=array([[64171,   893,   345, ..., 64171, 64171, 64171],
       [64171,   220,  4632, ..., 64171, 64171, 64171],
       [64171,   170,  1264, ..., 64171, 64171, 64171],
       ...,
       [64171,  1596,    88, ..., 64171, 64171, 64171],
       [64171,    51,  4842, ..., 64171, 64171, 64171],
       [64171,  3695,   369, ..., 64171, 64171, 64171]]), label_ids=array([[  893,   345,     6, ..., 64171, 64171, 64171],
       [  220,  2582,   130, ..., 64171, 64171, 64171],
       [  170,  1264,    11, ..., 64171, 64171, 64171],
       ...,
       [ 5893, 10767,     5, ..., 64171, 64171, 64171],
       [   51, 18406,   773, ..., 64171, 64171, 64171],
       [ 3695,   369, 12048, ..., 64171, 64171, 64171]]), metrics={'test_loss': 0.42624175548553467,

In [35]:
# Decode the predicted sequences
decoded_predictions_supara = tokenizer.batch_decode(final_test_results_supara.predictions, skip_special_tokens=True)
labels_supara = np.where(final_test_results_supara.label_ids != -100, final_test_results_supara.label_ids, tokenizer.pad_token_id)
decoded_labels_supara = tokenizer.batch_decode(labels_supara, skip_special_tokens=True)

# Print a sample of the predictions and labels
print("Sample Predictions and Labels:")
for i in range(10):  # Adjust the range to print more or fewer examples
    print(f"\nPrediction {i+1}: {decoded_predictions_supara[i]}")
    print(f"Label {i+1}: {decoded_labels_supara[i]}")

Sample Predictions and Labels:

Prediction 1: She likes red golf.
Label 1: She likes red roses.

Prediction 2: It strengthens our body.
Label 2: It makes our body firm.

Prediction 3: We live in Bangladesh.
Label 3: We live in Bangladesh.

Prediction 4: Human life is too short.
Label 4: Human life is very short.

Prediction 5: Traffic police have many responsibilities.
Label 5: Traffic police have heavy responsibilities.

Prediction 6: Cricket is my favorite game.
Label 6: Cricket is my favourite game.

Prediction 7: People should be careful.
Label 7: Mass awareness should be created.

Prediction 8: There may be some problems.
Label 8: There may be some difficulties.

Prediction 9: What's her name?
Label 9: What is her name?

Prediction 10: Our college is very old.
Label 10: Our college is very ancient.


In [36]:
# Save predictions to a CSV file without column names
predictions_df = pd.DataFrame({'Prediction': decoded_predictions_supara})
predictions_df.to_csv('results/helsinki_supara_predictions_mul_en.csv', index=False, header=False)

# Save labels to a separate CSV file without column names
labels_df = pd.DataFrame({'Label': decoded_labels_supara})
labels_df.to_csv('results/helsinki_supara_labels_mul_en.csv', index=False, header=False)